# Train Alert Recommendation Model

This notebook trains the KNN-based collaborative filtering model for alert recommendations.

**Inputs:**
- User data from MinIO
- Transaction data from MinIO

**Outputs:**
- Trained model saved as `model.pkl` in models directory
- Model metadata saved as `model_metadata.json`

In [ ]:
import os
import json
import pickle
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import boto3
from botocore.client import Config

In [ ]:
# Pipeline parameters
NAMESPACE = os.getenv('NAMESPACE', 'spending-transaction-monitor')
BUCKET_NAME = os.getenv('BUCKET_NAME', 'models')
DATA_VERSION = os.getenv('DATA_VERSION', '1')
MODEL_VERSION = os.getenv('MODEL_VERSION', '1.0.0')
N_NEIGHBORS = int(os.getenv('N_NEIGHBORS', '5'))
METRIC = os.getenv('METRIC', 'cosine')

# MinIO configuration (S3-compatible)
MINIO_ENDPOINT = os.getenv('MINIO_ENDPOINT', f'http://minio-service.{NAMESPACE}.svc.cluster.local:9000')
MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY', 'minio')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY', 'minio123')

print(f'Training configuration:')
print(f'  MinIO Endpoint: {MINIO_ENDPOINT}')
print(f'  Bucket: {BUCKET_NAME}')
print(f'  Data Version: {DATA_VERSION}')
print(f'  Model Version: {MODEL_VERSION}')
print(f'  N Neighbors: {N_NEIGHBORS}')
print(f'  Metric: {METRIC}')

In [ ]:
# Initialize MinIO client (S3-compatible)
s3_client = boto3.client(
    's3',
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY,
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)

print('✅ MinIO client initialized')

## Load Data from MinIO

In [ ]:
# Download data from MinIO
data_key_prefix = f'data/v{DATA_VERSION}'

# Create temp directory
os.makedirs('/tmp', exist_ok=True)

# Load users data
users_key = f'{data_key_prefix}/users.csv'
try:
    s3_client.download_file(BUCKET_NAME, users_key, '/tmp/users.csv')
    users_df = pd.read_csv('/tmp/users.csv')
    print(f'✅ Loaded {len(users_df)} users from MinIO')
except Exception as e:
    print(f'⚠️  Could not load users from MinIO: {e}')
    print(f'   Loading from local data directory instead...')
    users_df = pd.read_csv('./data/users.csv')
    print(f'✅ Loaded {len(users_df)} users from local data')

# Load transactions data
transactions_key = f'{data_key_prefix}/transactions.csv'
try:
    s3_client.download_file(BUCKET_NAME, transactions_key, '/tmp/transactions.csv')
    transactions_df = pd.read_csv('/tmp/transactions.csv')
    print(f'✅ Loaded {len(transactions_df)} transactions from MinIO')
except Exception as e:
    print(f'⚠️  Could not load transactions from MinIO: {e}')
    print(f'   Loading from local data directory instead...')
    transactions_df = pd.read_csv('./data/transactions.csv')
    print(f'✅ Loaded {len(transactions_df)} transactions from local data')

# Load existing alerts (if available)
try:
    alerts_key = f'{data_key_prefix}/user_alerts.csv'
    s3_client.download_file(BUCKET_NAME, alerts_key, '/tmp/user_alerts.csv')
    user_alerts_df = pd.read_csv('/tmp/user_alerts.csv')
    print(f'✅ Loaded {len(user_alerts_df)} alert preferences')
    use_real_alerts = True
except:
    print('ℹ️  No alert preferences found - will use heuristic labels')
    user_alerts_df = None
    use_real_alerts = False

## Feature Engineering

In [ ]:
def build_user_features(users_df, transactions_df):
    """Build behavioral features from transaction history"""
    
    # Convert amount to numeric
    transactions_df['amount'] = pd.to_numeric(transactions_df['amount'], errors='coerce')
    
    # Aggregate transaction metrics per user
    tx_agg = transactions_df.groupby('user_id').agg({
        'amount': ['count', 'mean', 'std', 'max', 'sum'],
        'merchant_name': pd.Series.nunique,
        'merchant_category': pd.Series.nunique
    })
    
    # Flatten column names
    tx_agg.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in tx_agg.columns]
    tx_agg = tx_agg.reset_index()
    tx_agg.columns = ['user_id', 'amount_count', 'amount_mean', 'amount_std', 
                      'amount_max', 'amount_sum', 'merchant_name_nunique', 
                      'merchant_category_nunique']
    
    # Join with user data
    user_feats = tx_agg.merge(
        users_df[['id', 'credit_limit', 'credit_balance']], 
        left_on='user_id', 
        right_on='id', 
        how='left'
    )
    
    if 'id' in user_feats.columns:
        user_feats = user_feats.drop(columns=['id'])
    
    # Calculate credit utilization
    user_feats['credit_limit'] = pd.to_numeric(user_feats['credit_limit'], errors='coerce').fillna(0)
    user_feats['credit_balance'] = pd.to_numeric(user_feats['credit_balance'], errors='coerce').fillna(0)
    user_feats['credit_utilization'] = np.where(
        user_feats['credit_limit'] > 0,
        user_feats['credit_balance'] / user_feats['credit_limit'],
        0
    )
    
    user_feats = user_feats.fillna(0)
    return user_feats

user_features = build_user_features(users_df, transactions_df)
print(f'✅ Built features for {len(user_features)} users')
print(f'   Features: {list(user_features.columns)}')

In [ ]:
# Alert type columns
ALERT_TYPES = [
    'alert_high_spender',
    'alert_high_tx_volume',
    'alert_high_merchant_diversity',
    'alert_near_credit_limit',
    'alert_large_transaction',
    'alert_new_merchant',
    'alert_location_based',
    'alert_subscription_monitoring'
]

def generate_heuristic_labels(df):
    """Generate alert labels based on behavioral heuristics"""
    df = df.copy()
    
    # High spender: top 25% by total spending
    df['alert_high_spender'] = (df['amount_sum'] >= df['amount_sum'].quantile(0.75)).astype(int)
    
    # High transaction volume: top 25% by transaction count
    df['alert_high_tx_volume'] = (df['amount_count'] >= df['amount_count'].quantile(0.75)).astype(int)
    
    # High merchant diversity: top 25% by unique merchants
    df['alert_high_merchant_diversity'] = (df['merchant_name_nunique'] >= df['merchant_name_nunique'].quantile(0.75)).astype(int)
    
    # Near credit limit: utilization >= 70%
    df['alert_near_credit_limit'] = (df['credit_utilization'] >= 0.7).astype(int)
    
    # Large transaction: max transaction > 75th percentile
    df['alert_large_transaction'] = (df['amount_max'] >= df['amount_max'].quantile(0.75)).astype(int)
    
    # Defaults for other types
    df['alert_new_merchant'] = 0
    df['alert_location_based'] = 0
    df['alert_subscription_monitoring'] = 0
    
    return df

if use_real_alerts and user_alerts_df is not None:
    # Use real alert labels
    pivot = user_alerts_df.pivot_table(
        index='user_id',
        columns='alert_type',
        values='enabled',
        fill_value=0,
        aggfunc='max'
    )
    pivot.columns = [f'alert_{col}' if not col.startswith('alert_') else col for col in pivot.columns]
    pivot = pivot.reset_index()
    
    user_features_with_labels = user_features.merge(pivot, on='user_id', how='left')
    for alert_type in ALERT_TYPES:
        if alert_type in user_features_with_labels.columns:
            user_features_with_labels[alert_type] = user_features_with_labels[alert_type].fillna(0).astype(int)
        else:
            user_features_with_labels[alert_type] = 0
    
    print('✅ Using real alert labels from data')
else:
    # Use heuristic labels
    user_features_with_labels = generate_heuristic_labels(user_features)
    print('✅ Generated heuristic-based alert labels')

# Show alert distribution
print('\n📊 Alert distribution:')
for alert_type in ALERT_TYPES:
    if alert_type in user_features_with_labels.columns:
        count = user_features_with_labels[alert_type].sum()
        pct = (count / len(user_features_with_labels)) * 100
        print(f'   {alert_type}: {int(count)} users ({pct:.1f}%)')

## Train KNN Model

In [ ]:
# Feature columns for similarity calculation
FEATURE_COLUMNS = [
    'amount_mean', 'amount_std', 'amount_max', 'amount_sum', 'amount_count',
    'merchant_name_nunique', 'merchant_category_nunique',
    'credit_limit', 'credit_balance', 'credit_utilization'
]

# Prepare training data
X = user_features_with_labels[FEATURE_COLUMNS].values
y = user_features_with_labels[ALERT_TYPES].values
user_ids = user_features_with_labels['user_id'].values

print(f'Training data shape:')
print(f'   X (features): {X.shape}')
print(f'   y (labels): {y.shape}')
print(f'   Users: {len(user_ids)}')

In [ ]:
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print('✅ Features normalized')

In [ ]:
# Train KNN model
knn_model = NearestNeighbors(
    n_neighbors=min(N_NEIGHBORS, len(X_scaled)),
    metric=METRIC,
    algorithm='brute'
)
knn_model.fit(X_scaled)

print(f'✅ KNN model trained')
print(f'   Algorithm: {knn_model.algorithm}')
print(f'   Metric: {knn_model.metric}')
print(f'   N neighbors: {knn_model.n_neighbors}')

## Save Model Artifacts

In [ ]:
# Create models directory
os.makedirs('models', exist_ok=True)

# Save model components
model_artifacts = {
    'knn_model': knn_model,
    'scaler': scaler,
    'feature_columns': FEATURE_COLUMNS,
    'alert_types': ALERT_TYPES,
    'user_ids': user_ids,
    'alert_labels': y,
    'training_features': X_scaled
}

with open('models/model.pkl', 'wb') as f:
    pickle.dump(model_artifacts, f)

print('✅ Model saved to models/model.pkl')

In [ ]:
# Save model metadata
metadata = {
    'model_version': MODEL_VERSION,
    'data_version': DATA_VERSION,
    'trained_at': datetime.now().isoformat(),
    'n_users': len(user_ids),
    'n_neighbors': N_NEIGHBORS,
    'metric': METRIC,
    'feature_columns': FEATURE_COLUMNS,
    'alert_types': ALERT_TYPES,
    'use_real_alerts': use_real_alerts,
    'model_size_bytes': os.path.getsize('models/model.pkl')
}

with open('models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print('✅ Metadata saved to models/model_metadata.json')
print(f'\nModel metadata:')
print(json.dumps(metadata, indent=2))

## Test Model

In [ ]:
# Test recommendation for a sample user
if len(user_ids) > 0:
    test_user_idx = 0
    test_user_id = user_ids[test_user_idx]
    test_features = X_scaled[test_user_idx:test_user_idx+1]
    
    # Find similar users
    distances, indices = knn_model.kneighbors(test_features, n_neighbors=N_NEIGHBORS)
    
    print(f'🧪 Test: Similar users to {test_user_id}:')
    for dist, idx in zip(distances[0], indices[0]):
        similar_user_id = user_ids[idx]
        print(f'   {similar_user_id}: distance={dist:.3f}')
    
    # Generate recommendations
    similar_labels = y[indices[0]]
    probabilities = similar_labels.mean(axis=0)
    
    print(f'\n📋 Recommended alerts for {test_user_id}:')
    for alert_type, prob in zip(ALERT_TYPES, probabilities):
        if prob >= 0.4:
            print(f'   {alert_type}: {prob:.2%} probability')

print('\n✅ Training notebook completed successfully!')
print('\nNext step: Run notebook 2 to save model to MinIO')